In [ ]:
# BASIC RENDERER

In [2]:
# IMPORTS
import numpy as np                              # For array manipulation
from PIL import Image                           # For image processing
import matplotlib.pyplot as plt                 # For applying colormaps
from sympy import sympify, lambdify, symbols    # For symbolic mathematics

In [73]:
# INITIALIZATION OF CONSTANTS
max_iterations = 256
max_magnitude = 2
attractor_str = 'z^2 + const' # initial definition of atractor function
const = -0.29609091 + 0.62491j 
res_w, res_h = 200, 200 # px
re_min, re_max, im_min, im_max = -0.9, 0.3, -0.3, 0.9 # complex plane range to be rendered

# lambda like callable compatible with numpy vectorized operations
attractor = lambda x1, x2: lambdify(symbols('z const'), sympify(attractor_str), 'numpy')(x1, x2)

In [57]:
# COMPUTING ORBITS
def if_in_Julia_set(z_arr:np.array, data:np.array, curr_const:complex=None, curr_iter:int=256):
    '''
    Calculates if Julia set contains a given point.
    Uses sympy expression for atractor function.
    
    Operates on passed data array !!!

    Parameters:
    - z_arr: array of complex numbers corresponding to pixels (np.array)
    - data: array to populate with iterations till exceeding max_magnitude or max_ieration if not exceeded (np.array)
    '''

    # adjustment for animated renders
    if not curr_const: curr_const = const
    max_iterations = curr_iter

    # initialize helper array
    not_exceeding = np.ones_like(data, dtype=bool)

    # iterate till exceeding max_magnitude or max_ieration if not exceeded 
    for _ in np.arange(max_iterations):

        # evaluate atractor function for relevant pixels, for current iteration
        z_arr = np.where(not_exceeding, attractor(z_arr, curr_const), z_arr)

        # mark points exceeding max_magnitude
        not_exceeding = ~(np.abs(z_arr) > max_magnitude)

        # update data
        data[not_exceeding] += 1

        # break the loop if all elements exceeded given magnitude
        if not any(not_exceeding): break

    # adjust data to prevent unint8 overflow
    data[data == max_iterations] = max_iterations-1
    

In [21]:
# RENDERING AND SAVING TO .png
def render_vectorwise(data:np.array) -> np.array:
    '''Renders Julia set as numpy array'''

    # initialize array of complex numbers corresponding to pixels
    # np.linspace creates array of evenly spaced numbers over resoluton range
    # np.newaxis adds new axis (column vector) to array
    # data contains complex numbers corresponding to pixels
    # max/min swaped beacuse rendering goes top to bottom
    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
            * np.linspace(im_max, im_min, res_h)[:, np.newaxis] 
    
    # calculate orbits
    if_in_Julia_set(z_arr, data)

def render(color_map:str="twilight_shifted") -> None:
    '''Renders Julia set into .png file'''

    # initialize image
    image = Image.new('RGB', (res_h, res_w))

    # initialize data
    data = np.zeros((res_h, res_w), dtype=np.uint8)

    # create data:
    render_vectorwise(data)

    # map data to colors
    # normalize orbits
    normalized_orbits = data / max_iterations
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * max_iterations).astype(np.uint8) # remove alpha channel

    # save data to image
    image = Image.fromarray(pixels, 'RGB')
    image.save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.png")

In [22]:
# EXECUTE
render()

In [23]:
# VISUAL ANLYSIS

In [24]:
# RENDERING SINGLE FRAME FOR ANIMATION
def render_frame(current_const:complex=0+0j, color_map:str="twilight_shifted") -> Image:
    '''Renders Julia set as numpy array'''

    data = np.zeros((res_h, res_w), dtype=np.uint8)

    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
        * np.linspace(im_max, im_min, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data, current_const)

    # map data to colors
    # normalize orbits
    normalized_orbits = data / max_iterations
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * max_iterations).astype(np.uint8) # remove alpha channel

    # return image
    return Image.fromarray(pixels, 'RGB')

In [25]:
# RENDERING INTO GIF FILE
def render_gif(frames_amount:int=200, frame_duration:int=25):

    magnitude = 0.8

    # const value list
    const_values = magnitude * np.exp(1j * np.linspace(0, 2 * np.pi, frames_amount))

    frames = []
    for i in range(frames_amount):
        
        curr_const = const_values[i]

        frames.append(render_frame(curr_const))

    frames[0].save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.gif", format='GIF', append_images=frames[1:], \
                    save_all=True, duration=frame_duration, loop=0)

In [26]:
# EXECUTE
render_gif(2,2)

In [27]:
# FRACTALS UNINTUITIVE DIMENTION

In [77]:
# RENDERING SINGLE FRAME FOR ANIMATION
def render_frame_uint16(current_iterations_max:int=256, color_map:str="twilight_shifted") -> Image:
    '''Renders Julia set as numpy array'''

    data = np.zeros((res_h, res_w), dtype=np.uint16)

    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
        * np.linspace(im_max, im_min, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data, const, current_iterations_max)

    # map data to colors
    # normalize orbits 
    normalized_orbits = data / current_iterations_max
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * 255).astype(np.uint8) # remove alpha channel

    # return image
    return Image.fromarray(pixels, 'RGB')

In [78]:
# RENDERING INTO GIF FILE
def render_gif(frames_amount:int=500, frame_duration:int=25, log_2_max_iter_start:int=4, log_2_max_iter_end:int=12):

    # spread vaalues of max iter logarithmically to better see first elements
    # log_2_max_iter_start  - start of logarythmic scale (2^log_...)
    # log_2_max_iter_end    - end of logarythmic scale (2^log_...)
    # frames_amount         - how many elements to generate
    # False                 - not including log_2_max_iter_end as element (ensures not getting to limit of uint16 data type)
    # int                   - casts elements too ints
    max_iter_values = np.logspace(log_2_max_iter_start, log_2_max_iter_end, frames_amount, False, 2, int)
    
    frames = []
    for i in range(frames_amount):
        
        curr_iter = max_iter_values[i]

        frames.append(render_frame_uint16(curr_iter, "twilight"))

    frames[0].save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.gif", format='GIF', append_images=frames[1:], \
                    save_all=True, duration=frame_duration, loop=0)
    


In [ ]:
# EXECUTE
render_gif(10, 500)